In [1]:
# Import Cell
import pandas as pd
import datetime
import pprint as pp
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from scipy import stats
from sklearn.metrics import r2_score
import numpy as np
from datetime import datetime
date_format = "%Y-%m-%d"

In [2]:
# Functions

def executeCLF(clf, x_train, y_train, x_test, y_test):
    clf.fit(x_train.values, y_train.values)
    test_preds = clf.predict(x_test.values)
    train_preds = clf.predict(x_train.values)
    print("Accuracy on train set: " + str(accuracy_score(y_train, train_preds))) 
    print("Accuracy on test set: " + str(accuracy_score(y_test, test_preds))) 
    return pd.DataFrame({'PREDICTION': test_preds}), accuracy_score(y_test, test_preds)

def printDetailedPreds(preds_df, y_test):
    detailed_preds = pd.concat([preds_df.reset_index(drop=True), y_test.reset_index(drop=True)])
    return detailed_preds

In [19]:
df = pd.read_csv('../data/outputs/final_norm.csv')
df1 = pd.read_csv('../data/outputs/ranks.csv')
game_id = df1[['GAME_ID_HOME']]

df1_norm = df1.drop(['GAME_ID_HOME'],axis=1)
df1_norm = (df1_norm - df1_norm.mean()) / (df1_norm.max() - df1_norm.min())
df1 = pd.concat([game_id, df1_norm], axis=1)
df = pd.merge(df, df1, left_on=['CURRENT_GAME_ID'], right_on=['GAME_ID_HOME'])

train_start_date = datetime.strptime("2016-10-26", date_format)
train_end_date = datetime.strptime("2017-10-17", date_format)
test_date = datetime.strptime("2017-10-18", date_format)

date_list = df["GAME_DATE_EST"][0:3269].drop_duplicates()

test_end_index = int(df.index[df['GAME_DATE_EST'] == "2018-04-10"][-1])
test_start_index = int(df.index[df['GAME_DATE_EST'] == "2017-10-18"][0])
train_end_index = int(df.index[df['GAME_DATE_EST'] == "2016-11-29"][-1])
train_start_index = int(df.index[df['GAME_DATE_EST'] == "2017-04-08"][0])

temp = pd.get_dummies(df[['TEAM_NAME_AWAY']])
df = df.drop(columns=['TEAM_NAME_AWAY'])
df = pd.concat([df, temp], axis=1)

temp = pd.get_dummies(df[['TEAM_NAME_HOME']])
df = df.drop(columns=['TEAM_NAME_HOME'])
df = pd.concat([df, temp], axis=1)
df = df.drop(['GAME_DATE_EST','CURRENT_GAME_ID', 'GAME_ID_HOME'],axis=1)

In [4]:
#Logistic Regression
y = df['WIN_HOME']
x = df.drop(columns=['WIN_HOME'])
print("\nResults for Logistic Regression model:")
skf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
accuracies = []
count = 1
for train_index, test_index in skf.split(x, y):  
    print('FOLD ' + str(count) + ": ")
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  
    log_clf = LogisticRegression(solver='lbfgs')
    log_preds_df, accuracy = executeCLF(log_clf, x_train, y_train, x_test, y_test)
    accuracies.append(accuracy)
    print("\n")
    count+=1
print('AVG across folds: ' + str(np.mean(accuracies)))


Results for Logistic Regression model:
FOLD 1: 
Accuracy on train set: 0.6953045253487581
Accuracy on test set: 0.7125382262996942


FOLD 2: 
Accuracy on train set: 0.7094249744811161
Accuracy on test set: 0.6422018348623854


FOLD 3: 
Accuracy on train set: 0.7068730860837019
Accuracy on test set: 0.6773700305810397


FOLD 4: 
Accuracy on train set: 0.7007485539299081
Accuracy on test set: 0.6865443425076453


FOLD 5: 
Accuracy on train set: 0.6980265396393331
Accuracy on test set: 0.6529051987767585


FOLD 6: 
Accuracy on train set: 0.7017693092888738
Accuracy on test set: 0.7048929663608563


FOLD 7: 
Accuracy on train set: 0.6981292517006803
Accuracy on test set: 0.7223926380368099


FOLD 8: 
Accuracy on train set: 0.6979591836734694
Accuracy on test set: 0.7208588957055214


FOLD 9: 
Accuracy on train set: 0.6943877551020409
Accuracy on test set: 0.7162576687116564


FOLD 10: 
Accuracy on train set: 0.7042517006802721
Accuracy on test set: 0.696319018404908


AVG across folds: 0.

In [20]:
# Train test split as 2017-2018 season
y = df['WIN_HOME']
x = df.drop(columns=['WIN_HOME'])
print("\nResults for Logistic Regression model:")

x_train, x_test = x.iloc[train_start_index:train_end_index], x.iloc[test_start_index:test_end_index]
y_train, y_test = y.iloc[train_start_index:train_end_index], y.iloc[test_start_index:test_end_index]
  
logistic_reg = LogisticRegression(solver='lbfgs')
log_preds_df, accuracy = executeCLF(logistic_reg, x_train, y_train, x_test, y_test)
accuracies.append(accuracy)
print("\n")


Results for Logistic Regression model:


ValueError: Found array with 0 sample(s) (shape=(0, 68)) while a minimum of 1 is required.

In [ ]:
#Backward Feature Selection 

train_x = x_train
train_y = y_train
test_x = x_test
test_y = y_test

# Initial calculation of p values
pvalues = stats.ttest_ind(train_x, train_y)[1]

best_logistic_r2 = 0
best_logistic_model = logistic_reg
best_logistic_x_train = train_x
best_logistic_x_test = test_x

while len(pvalues) > 1 :
    # Drop highest p value
    m = max(pvalues)
    m_index = [i for i, j in enumerate(pvalues) if j == m]
    train_x = train_x.drop(train_x.columns[m_index[0]], axis = 1)
    test_x = test_x.drop(test_x.columns[m_index[0]], axis = 1)

    # Recalculate p values after drop
    pvalues = stats.ttest_ind(train_x, train_y)[1]
    
    # Fit model
    new_logistic_reg = LogisticRegression(solver='lbfgs').fit(train_x.values, train_y.values)
        
    # Replace logistic model if new model is better 
    new_test_preds = new_logistic_reg.predict(test_x.values)  
    new_logistic_r2 = accuracy_score(test_y, new_test_preds)
    if new_logistic_r2 > best_logistic_r2:
        best_logistic_r2 = new_logistic_r2
        best_logistic_model = new_logistic_reg
        best_logistic_x_train = train_x
        best_logistic_x_test = test_x
# Results of best models
new_test_preds = best_logistic_model.predict(best_logistic_x_test.values)
print("\nResults after feature selection for logistic model:")
print("Accuracy on test set: " + str(accuracy_score(test_y, new_test_preds))) 

In [ ]:
# Random Forest
y = df['WIN_HOME']
x = df.drop(columns=['WIN_HOME'])
print("\nResults for Random Forest model:")
skf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
accuracies = []
count = 1
for train_index, test_index in skf.split(x, y):  
    print('FOLD ' + str(count) + ": ")
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  
    rf_clf = RandomForestClassifier(n_estimators=20, max_depth=10)
    rf_preds_df, accuracy = executeCLF(rf_clf, x_train, y_train, x_test, y_test)
    accuracies.append(accuracy)
    print("\n")
    count+=1
print('AVG across folds: ' + str(np.mean(accuracies)))

In [12]:
# Train test split as 2017-2018 season
y = df['WIN_HOME']
x = df.drop(columns=['WIN_HOME'])
print("\nResults for Random Forest model:")

x_train, x_test = x.iloc[train_start_index:train_end_index], x.iloc[test_start_index:test_end_index]
y_train, y_test = y.iloc[train_start_index:train_end_index], y.iloc[test_start_index:test_end_index]
for i in range(0, 10):
    rf_reg = RandomForestClassifier(n_estimators=100, max_depth=11)
    rf_preds_df, accuracy = executeCLF(rf_reg, x_train, y_train, x_test, y_test)
    accuracies.append(accuracy)
    print("\n")


Results for Random Forest model:


ValueError: Found array with 0 sample(s) (shape=(0, 68)) while a minimum of 1 is required.

In [ ]:
# ADABoost 
y = df['WIN_HOME']
x = df.drop(columns=['WIN_HOME'])
print("\nResults for ADABoost model:")
skf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
accuracies = []
count = 1
for train_index, test_index in skf.split(x, y):  
    print('FOLD ' + str(count) + ": ")
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  
    rf_clf = AdaBoostClassifier(n_estimators=20)
    rf_preds_df, accuracy = executeCLF(rf_clf, x_train, y_train, x_test, y_test)
    accuracies.append(accuracy)
    print("\n")
    count+=1
print('AVG across folds: ' + str(np.mean(accuracies)))

In [ ]:
# AdaBoost
y = df['WIN_HOME']
x = df.drop(columns=['WIN_HOME'])
print("\nResults for AdaBoost model:")

x_train, x_test = x.iloc[train_start_index:train_end_index], x.iloc[test_start_index:test_end_index]
y_train, y_test = y.iloc[train_start_index:train_end_index], y.iloc[test_start_index:test_end_index]
  
rf_reg = AdaBoostClassifier(n_estimators=20)
rf_preds_df, accuracy = executeCLF(rf_reg, x_train, y_train, x_test, y_test)
accuracies.append(accuracy)
print("\n")